In [1]:
import os
import pandas as pd

print("Loadind Dataset: gestures")
path = 'gestures-dataset'

dataset = None
dataset_idx = None
samples = 0
for subject in os.listdir(path):
    if os.path.isfile(os.path.join(path, subject)):
        continue
    if subject in ('U01', 'U02', 'U03', 'U04', 'U05', 'U06', 'U07', 'U08'):
        for gesture in os.listdir(os.path.join(path, subject)):
            if os.path.isfile(os.path.join(path, subject, gesture)):
                continue
            gesture = str(gesture)
            #if gesture not in gesture_subset:
            #    continue
            for samplefile in os.listdir(os.path.join(path, subject, gesture)):
                if os.path.isfile(os.path.join(path, subject, gesture, samplefile)):
                    df = pd.read_csv(os.path.join(path, subject, gesture, samplefile), \
                        sep = ' ', \
                        names = ['System.currentTimeMillis()', \
                        'System.nanoTime()', \
                        'sample.timestamp', \
                        'X', \
                        'Y', \
                        'Z' \
                        ])
                    df = df[["sample.timestamp", "X", "Y", "Z"]]
                                        
                    start = df["sample.timestamp"][0]
                    df["sample.timestamp"] -= start
                    df["sample.timestamp"] /= 10000000
                    df["subject"] = subject
                    df["gesture"] = gesture
                    df["sample"] = str(samplefile[:-4])
                    #print(df)
                    if dataset is None:
                        dataset = df.copy()
                    else:
                        dataset = pd.concat([dataset, df])

                    df_idx_sample = pd.DataFrame()
                    df_idx_sample = df_idx_sample.append({'subject':subject, 'gesture': gesture, 'sample': str(samplefile[:-4])}, ignore_index = True)
                    if dataset_idx is None:
                        dataset_idx = df_idx_sample.copy()
                    else:
                        dataset_idx = pd.concat([dataset_idx, df_idx_sample])
                    samples += 1

dataset = dataset.sort_values(by=['gesture','subject','sample','sample.timestamp'])
dataset_idx = dataset_idx.sort_values(by=['gesture','subject','sample'])
data = dataset  
print(str(samples) + " samples loaded")

print("Scaling Dataset: gestures")
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
dataset_scaled = None
samples = 0
#for i, gesture in enumerate(gesture_subset):
for i, gesture in enumerate(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']):
    df_gesture=data[data['gesture']==gesture] 
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject=df_gesture[df_gesture['subject']==subject]
        for k, sample in enumerate(df_subject['sample'].unique()):
            df_sample=df_subject[df_subject['sample']==sample].copy()
            df_sample.sort_values(by=['sample.timestamp'])

            sc = scaler
            sc = sc.fit_transform(df_sample[["X", "Y", "Z"]])
            sc = pd.DataFrame(data=sc, columns=["X", "Y", "Z"])
            df_sample['X'] = sc['X']
            df_sample['Y'] = sc['Y']
            df_sample['Z'] = sc['Z']
            if dataset_scaled is None:
                dataset_scaled = df_sample.copy()
            else:
                dataset_scaled = pd.concat([dataset_scaled, df_sample])
            samples += 1
print(str(samples) + " samples scaled")
data = dataset_scaled

Loadind Dataset: gestures
3251 samples loaded
Scaling Dataset: gestures
3251 samples scaled


In [2]:
import tsfel

cfg_file = tsfel.get_features_by_domain(domain=None, json_path="features.json")
#cfg_file = tsfel.get_features_by_domain("statistical")
#cfg_file = tsfel.get_features_by_domain("temporal")
#cgf_file = tsfel.get_features_by_domain("spectral")

print("Extracting All Features as describe in features.json, total of 384, 128 per axis: dataset_scaled")
data = dataset_scaled
print("Set: " + str(len(data.index)) + " measurements")
dataset_features = None
samples = 0
for i, gesture in enumerate(data['gesture'].unique()):
    df_gesture = data[data['gesture']==gesture]
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject = df_gesture[df_gesture['subject']==subject]
        for k, sample in enumerate(df_subject['sample'].unique()):
            df_sample = df_subject[df_subject['sample']==sample]
            df_sample.sort_values(by=['sample.timestamp'])

            df_feature = pd.DataFrame(columns = ["gesture","subject", "sample"])
            #print(df_sample["Y"].size)
            df_feature = df_feature.append({ \
                                        'gesture' :gesture, 'subject' : subject, 'sample' : sample, \
                                        'features': pd.concat( \
                                            [
                                                tsfel.time_series_features_extractor(cfg_file, df_sample["X"], fs=1000, verbose=0), \
                                                tsfel.time_series_features_extractor(cfg_file, df_sample["Y"], fs=1000, verbose=0), \
                                                tsfel.time_series_features_extractor(cfg_file, df_sample["Z"], fs=1000, verbose=0)  \
                                            ], axis = 1) \
                                       }, \
                                       ignore_index=True)
            if dataset_features is None:
                dataset_features = df_feature.copy()
            else:
                dataset_features = pd.concat([dataset_features, df_feature], ignore_index=True)
            samples += 1
#print(dataset_features.head(10))
#print(dataset_features.tail(10))
print("Features extracted from " + str(samples) + " samples")
dataset_scaled_features = dataset_features


Extracting All Features as describe in features.json, total of 384, 128 per axis: dataset_scaled
Set: 64070 measurements
Features extracted from 3251 samples


In [3]:
print("Segregating outliers for testing separately: gestures")
data = dataset_scaled
dataset_outliers = None
dataset_outliers_idx = None
dataset_cleaned = None
dataset_cleaned_idx = None

samples = 0
outliers = 0
#for i, gesture in enumerate(gesture_subset):
for i, gesture in enumerate(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']):
    df_gesture = data[data['gesture']==gesture]
    for j, subject in enumerate(df_gesture['subject'].unique()):
        df_subject = df_gesture[df_gesture['subject']==subject]
        
        time_mean = df_subject.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['mean']})
        time_std = df_subject.groupby(["gesture","subject", "sample"]).count().groupby(["gesture","subject"]).agg({'sample.timestamp': ['std']})
        time_max = time_mean['sample.timestamp'].iloc[0]['mean'] + 1.0 * time_std['sample.timestamp'].iloc[0]['std']
        time_min = time_mean['sample.timestamp'].iloc[0]['mean'] - 1.0 * time_std['sample.timestamp'].iloc[0]['std']
        for k, sample in enumerate(df_subject['sample'].unique()):
            df_sample=df_subject[df_subject['sample']==sample]
            df_sample_count = df_sample.count()['sample.timestamp']
            if df_sample_count < time_min or df_sample_count > time_max:
                if dataset_outliers is None:
                    dataset_outliers = df_sample.copy()
                else:
                    dataset_outliers = pd.concat([dataset_outliers, df_sample])
                df_idx_sample = pd.DataFrame()
                df_idx_sample = df_idx_sample.append({'subject':subject, 'gesture': gesture, 'sample': sample}, ignore_index = True)
                if dataset_outliers_idx is None:
                    dataset_outliers_idx = df_idx_sample.copy()
                else:
                    dataset_outliers_idx = pd.concat([dataset_outliers_idx, df_idx_sample])
                outliers += 1
            else:
                if dataset_cleaned is None:
                    dataset_cleaned = df_sample.copy()
                else:
                    dataset_cleaned = pd.concat([dataset_cleaned, df_sample])
                df_idx_sample = pd.DataFrame()
                df_idx_sample = df_idx_sample.append({'subject':subject, 'gesture': gesture, 'sample': sample}, ignore_index = True)
                if dataset_cleaned_idx is None:
                    dataset_cleaned_idx = df_idx_sample.copy()
                else:
                    dataset_cleaned_idx = pd.concat([dataset_cleaned_idx, df_idx_sample])
                samples += 1
print(str(samples) + " samples cleaned")
print(str(outliers) + " samples outliers")
data = dataset_cleaned

Segregating outliers for testing separately: gestures
2479 samples cleaned
772 samples outliers


In [4]:
print("Segregating a stratified test set in addition to outliers: gestures")
from sklearn.model_selection import train_test_split
data = dataset_cleaned_idx
train, test = train_test_split(data, test_size=0.15, train_size=0.85, random_state=1000, shuffle=True, stratify=data['gesture'])
print("Training set: " + str(len(train.index)) + " samples")
print("Test set: " + str(len(test.index)) + " samples")

Segregating a stratified test set in addition to outliers: gestures
Training set: 2107 samples
Test set: 372 samples


In [5]:
print("Filling training samples with measurement data")
data = dataset_cleaned
dataset_cleaned_train = None
samples = 0
for tmp in train.iterrows():
    df = data[data['gesture'] == tmp[1]['gesture']]
    df = df[df['subject'] == tmp[1]['subject']]
    df = df[df['sample'] == tmp[1]['sample']]
    if dataset_cleaned_train is None:
        dataset_cleaned_train = df.copy()
    else:
        dataset_cleaned_train = pd.concat([dataset_cleaned_train, df])
    samples += 1
print(str(samples) + " training samples filled")

print("Filling test samples with measurement data")
data = dataset_cleaned
dataset_cleaned_test = None
samples = 0
for tmp in test.iterrows():
    df = data[data['gesture'] == tmp[1]['gesture']]
    df = df[df['subject'] == tmp[1]['subject']]
    df = df[df['sample'] == tmp[1]['sample']]
    if dataset_cleaned_test is None:
        dataset_cleaned_test = df.copy()
    else:
        dataset_cleaned_test = pd.concat([dataset_cleaned_test, df])
    samples += 1
print(str(samples) + " test samples filled")



Filling training samples with measurement data
2107 training samples filled
Filling test samples with measurement data
372 test samples filled


In [6]:
print("Filling training samples with features data")
data = dataset_scaled_features
dataset_cleaned_train_features = None
samples = 0
for tmp in train.iterrows():
    df = data[data['gesture'] == tmp[1]['gesture']]
    df = df[df['subject'] == tmp[1]['subject']]
    df = df[df['sample'] == tmp[1]['sample']]
    if dataset_cleaned_train_features is None:
        dataset_cleaned_train_features = df.copy()
    else:
        dataset_cleaned_train_features = pd.concat([dataset_cleaned_train_features, df])
    samples += 1
print(str(samples) + " training samples filled")

print("Filling test samples with feature data")
data = dataset_scaled_features
dataset_cleaned_test_features = None
samples = 0
for tmp in test.iterrows():
    df = data[data['gesture'] == tmp[1]['gesture']]
    df = df[df['subject'] == tmp[1]['subject']]
    df = df[df['sample'] == tmp[1]['sample']]
    if dataset_cleaned_test_features is None:
        dataset_cleaned_test_features = df.copy()
    else:
        dataset_cleaned_test_features = pd.concat([dataset_cleaned_test_features, df])
    samples += 1
print(str(samples) + " test samples filled")

print("Filling outliers samples with feature data")
data = dataset_scaled_features
dataset_outliers_features = None
samples = 0
for tmp in dataset_outliers_idx.iterrows():
    df = data[data['gesture'] == tmp[1]['gesture']]
    df = df[df['subject'] == tmp[1]['subject']]
    df = df[df['sample'] == tmp[1]['sample']]
    if dataset_cleaned_test_features is None:
        dataset_outliers_features = df.copy()
    else:
        dataset_outliers_features = pd.concat([dataset_outliers_features, df])
    samples += 1
print(str(samples) + " outliers samples filled")



Filling training samples with features data
2107 training samples filled
Filling test samples with feature data
372 test samples filled
Filling outliers samples with feature data
772 outliers samples filled


In [8]:
data = dataset_cleaned_train_features
work = data.copy()
work = work.drop(labels = ["gesture","subject", "sample"], axis = 1)
work = pd.DataFrame(work['features'])
print(work)

data = dataset_cleaned_test_features
work = data.copy()
work = work.drop(labels = ["gesture","subject", "sample"], axis = 1)
work = pd.DataFrame(work['features'])
print(work)

data = dataset_outliers_features
work = data.copy()
work = work.drop(labels = ["gesture","subject", "sample"], axis = 1)
work = pd.DataFrame(work['features'])
print(work)



                                               features
220      0_FFT mean coefficient_0  0_FFT mean coeffi...
610      0_FFT mean coefficient_0  0_FFT mean coeffi...
293      0_FFT mean coefficient_0  0_FFT mean coeffi...
2776     0_FFT mean coefficient_0  0_FFT mean coeffi...
2183     0_FFT mean coefficient_0  0_FFT mean coeffi...
...                                                 ...
622      0_FFT mean coefficient_0  0_FFT mean coeffi...
705      0_FFT mean coefficient_0  0_FFT mean coeffi...
1972     0_FFT mean coefficient_0  0_FFT mean coeffi...
1007     0_FFT mean coefficient_0  0_FFT mean coeffi...
820      0_FFT mean coefficient_0  0_FFT mean coeffi...

[2107 rows x 1 columns]
                                               features
851      0_FFT mean coefficient_0  0_FFT mean coeffi...
3162     0_FFT mean coefficient_0  0_FFT mean coeffi...
701      0_FFT mean coefficient_0  0_FFT mean coeffi...
1788     0_FFT mean coefficient_0  0_FFT mean coeffi...
1534     0_FFT mean coe

In [11]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline


data = dataset_cleaned_train_features
work = data.copy()
targets = pd.DataFrame(work[["gesture","subject","sample"]], columns=["gesture","subject","sample"])
targets.reset_index(inplace=True, drop=True)
target_for_plot = pd.DataFrame(work[["gesture"]], columns=["gesture"])
target_for_plot.reset_index(inplace=True, drop=True)
work = work.drop(labels = ["gesture","subject", "sample"], axis = 1)
print(work.values)

#pca = PCA(n_components=3)
pca = PCA(.99)
principalComponents = pca.fit_transform(work)
#print(principalComponents[0])
#print(pca.explained_variance_ratio_)
#total = 0
#for tmp in pca.explained_variance_ratio_:
#    total += tmp
#print(total)
print(pca.n_features_)
print(pca.n_samples_)
#print(pca.noise_variance_)
print(pca.n_components_)
#print(pca.components_)

columns = []
for i in range(pca.n_components_):
    columns.append("principal component "+str(i+1))
features_pca = pd.DataFrame(data = principalComponents, columns = columns)
features_pca = pd.concat([features_pca, targets], axis = 1)
features_pca_for_plot = pd.DataFrame(data = principalComponents, columns = columns)
features_pca_for_plot = pd.concat([features_pca_for_plot, target_for_plot], axis = 1)

fig = plt.figure(figsize = (70,70))
import seaborn as sns

sns.set_style("whitegrid");
#sns.pairplot(iris, hue = "species", size = 3);
#plt.show()
# NOTE: the diagonal elements are PDFs for each feature.import seaborn as sns
sns.pairplot(features_pca_for_plot, hue='gesture')
plt.show()

[[   0_FFT mean coefficient_0  0_FFT mean coefficient_1  \
  0              4.242484e-07                  0.001894

     0_FFT mean coefficient_2  0_FFT mean coefficient_3  \
  0                  0.004739                  0.002492

     0_FFT mean coefficient_4  0_FFT mean coefficient_5  \
  0                  0.001713                  0.002415

     0_FFT mean coefficient_6  0_Fundamental frequency  0_Human range energy  \
  0                  0.000964                    200.0                   0.0

      0_MFCC_0  ...  0_Median diff  0_Negative turning points  \
  0 -73.168379  ...      -0.018173                        2.0

     0_Neighbourhood peaks  0_Peak to peak distance  0_Positive turning points  \
  0                    0.0                  3.90732                        3.0

     0_Signal distance   0_Slope  0_Sum absolute diff  0_Total energy  \
  0          16.792881 -0.080256             8.069069     1090.909091

     0_Zero crossing rate
  0                   2.0

  [1 ro

ValueError: setting an array element with a sequence.